# Get list of stories from listing files

In [881]:
import json
import os

from bs4 import BeautifulSoup
from datetime import date, datetime
import lxml.html
from pathlib import Path

In [882]:
def get_all_files(dir_path):
    # get list of all files in dir
    dir_list = os.listdir(dir_path)

#     print("Files and directories in '", path, "' :")
#     print(dir_list)
    return dir_list

dir_path = "./list-pages/"
files_list = get_all_files(dir_path)

In [883]:
dest = files_list[0]
fpath = f"{dir_path}{dest}"
fpath

'./list-pages/2023-07-22__orderby=date&_page=29.html'

In [357]:
def parse_listings_get_story_links(fpath):
    file = open(fpath, "r")
    page_html = file.read()
    file.close()

    soup = BeautifulSoup(page_html, "lxml")
    soup.title
    story_tags = soup.select("a.pt-cv-readmore")

    return [a["href"] for a in story_tags]

In [358]:
# test
story_links = parse_listings_get_story_links(fpath)
# story_links

In [359]:
dir_path = "./list-pages/"
files_list = get_all_files(dir_path)

In [360]:
def get_all_story_links(files_list, dir_path):
    all_story_links = {}

    for fpath in files_list:
        if fpath == ".DS_Store":
            continue
#         print(fpath)
        all_story_links[fpath] = parse_listings_get_story_links(f"{dir_path}{fpath}")

    return all_story_links

In [657]:
story_links_dict = get_all_story_links(files_list, dir_path)
story_links_dict

{'2023-07-22__orderby=date&_page=29.html': ['https://www.creepypasta.com/the-scariest-story-ever/',
  'https://www.creepypasta.com/windows-and-notes/',
  'https://www.creepypasta.com/sick-2/',
  'https://www.creepypasta.com/a-funny-kind-of-day/',
  'https://www.creepypasta.com/the-strangest-security-tape-ive-ever-seen/',
  'https://www.creepypasta.com/lashes/',
  'https://www.creepypasta.com/the-raccoon/',
  'https://www.creepypasta.com/ptsd/',
  'https://www.creepypasta.com/never-again/',
  'https://www.creepypasta.com/bugs/',
  'https://www.creepypasta.com/typing/',
  'https://www.creepypasta.com/homicidal-tendencies/',
  'https://www.creepypasta.com/spectrophobia/',
  'https://www.creepypasta.com/the-painting/',
  'https://www.creepypasta.com/the-sixth-floor/',
  'https://www.creepypasta.com/very-ape/',
  'https://www.creepypasta.com/lita-a-bak/',
  'https://www.creepypasta.com/shadows/',
  'https://www.creepypasta.com/footprints/',
  'https://www.creepypasta.com/the-ritual/',
  'ht

In [436]:
with open(f"./parsed-listings/{date.today()}_story_links.json", "w") as outfile:
    json.dump(story_links_dict, outfile, sort_keys=True)

## Get Stories

In [658]:
import json
import requests

from datetime import date
from pathlib import Path
from time import sleep

In [659]:
stories_list_fpath = "./parsed-listings/2023-07-23_story_links.json"
with open(stories_list_fpath, "r") as stories_list_file:
    stories_list_dict = json.load(stories_list_file)

In [660]:
for k in stories_list_dict.keys():
    fetch_pages(k, stories_list_dict[k])
#     print(stories_list_dict[k])
    break # testing one round

Already have story-pages/published_stories_contamination.html, loading!
Already have story-pages/published_stories_holder-of-eternity.html, loading!
Already have story-pages/published_stories_theo-twining.html, loading!
Already have story-pages/published_stories_the-town-of-blanche.html, loading!
Already have story-pages/published_stories_the-church-cellar.html, loading!
Already have story-pages/published_stories_the-toolbox.html, loading!
Already have story-pages/published_stories_a-chat-over-dinner.html, loading!
Already have story-pages/published_stories_your-life-is-a-movie.html, loading!
Already have story-pages/published_stories_the-seven-springs.html, loading!
Already have story-pages/published_stories_the-photograph-pile.html, loading!
Already have story-pages/published_stories_the-forgotten-vending-machine.html, loading!
Already have story-pages/published_stories_the-hooded-being.html, loading!
Already have story-pages/published_stories_in-the-mirror.html, loading!
Already hav

In [884]:
# RATINGS AND REVIEWS DYNAMICALLY LOADED
def fetch_page(url):
    ident = (
    "Stephanie Andrews (jellomoat@gmail.com), " + 
    "scraping for educational purposes"
    )
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
    }

    print(f"Fetching {url}")
    return requests.get(
        url,
        headers=headers
    )

In [885]:
url = "https://www.creepypasta.com/i-watch-security-cameras-in-an-old-government-building-part-one/"
fetch_page(url)

Fetching https://www.creepypasta.com/i-watch-security-cameras-in-an-old-government-building-part-one/


<Response [200]>

In [886]:
def fetch_pages(list_page, links_list):
    # fetch each page
    for link in links_list:
        dest = Path("story-pages/" + f"published_stories_{link.split('/')[-2]}.html")

        if dest.exists(): # load it from file
            print(f"Already have {dest}, loading!")
            file = open(dest, "r")
            page_html = file.read()
            file.close()

        else: # fetch it!
            page_resp = fetch_page(link)
            page_html = page_resp.text
            sleep(2)

            # save to file
            with open(dest, "w") as f:
                f.write(page_html)
            story_writing_log(list_page, dest)

    return None

In [887]:
def story_writing_log(listing_fname="testing", dest_fname="testing", logname="stories_log"):
    with open(f"{date.today()}_{logname}.txt", "a") as f:
        f.write(f"{listing_fname}, {dest_fname}\n")

In [888]:
links_list = [
    "https://www.creepypasta.com/contamination/",
    "https://www.creepypasta.com/holder-of-eternity/",
    "https://www.creepypasta.com/theo-twining/",]

In [889]:
# fetch per list-pages file, replace params
fetch_pages(list_page="test-list-page", links_list=links_list)

Already have story-pages/published_stories_contamination.html, loading!
Already have story-pages/published_stories_holder-of-eternity.html, loading!
Already have story-pages/published_stories_theo-twining.html, loading!


# Parse Stories

In [890]:
import re

from bs4 import BeautifulSoup
from datetime import datetime
import lxml.html

In [891]:
def get_html(fpath):
    file = open(fpath, "r")
    page_html = file.read()
    file.close()
    return page_html

In [892]:
page_html = get_html("./story-pages/published_stories_contamination.html")
soup = BeautifulSoup(page_html, "lxml")
date_str = get_date(soup)

In [893]:
def get_date(soup):
    raw_date = soup.select_one("span.published").string.strip(" ")
    date_str = datetime.strftime(datetime.strptime(raw_date, "%B %d, %Y"), "%Y-%m-%d")
    return date_str

In [894]:
story_text, author_str = get_story_and_author_content(soup)
author_str
# story_text

'Alice Wilde'

In [895]:
def get_reading_time_min(soup):
    reading_time_el = soup.select_one("span.rt-reading-time")
    reading_time_raw_unit = reading_time_el.select_one("span.rt-postfix").string
    reading_time_raw_unit

    reading_time_raw = int(reading_time_el.select_one("span.rt-time").string.strip(" "))
    reading_time_min = reading_time_raw if re.search(r"minutes*", reading_time_raw_unit) else (reading_time_raw * 60)
    return reading_time_min

In [896]:
get_reading_time_min(soup)

2

In [897]:
def get_categories(tag):
    return tag.has_attr('href') \
        and re.search(r"category", tag["href"]) \
        and not re.search(r"archive", tag["href"])

def get_tags(tag):
    return tag.has_attr('href') \
        and re.search(r"tag", tag["href"]) \
        and not re.search(r"instagram", tag["href"])

In [898]:
story_cats = soup.find_all(get_categories)
story_tags = soup.find_all(get_tags)

In [899]:
def get_story_uuid(soup):
    return soup.select_one("article")["id"].strip("post-")
#     return soup.select_one("input#comment_post_ID")["value"]

In [900]:
story_uuid = get_story_uuid(soup)
story_uuid

'210'

In [901]:
def get_html(fpath):
    file = open(fpath, "r")
    page_html = file.read()
    file.close()
    return page_html


class CreepyPastaSoupParser:
    def __init__(self, fpath_or_link, is_fpath=True):
        self.fpath_or_link = fpath_or_link
        self.soup = self.get_soup(is_fpath)
        self.title = self.soup.title.string        

    def get_soup(self, is_fpath):
        if is_fpath:
            page_html = get_html(self.fpath_or_link)
        else:
            page_html = fetch_page(self.fpath_or_link).text
        return BeautifulSoup(page_html, "lxml")
        
    def parse_story(self):
        parsed_story_dict = {}
        parsed_story_dict["title"] = self.title
        parsed_story_dict["link"] = self.story_link
        parsed_story_dict["date_str"] = self.published_date
        parsed_story_dict["story_text"], parsed_story_dict["author_str"] = self.__get_story_and_author_content()
        parsed_story_dict["reading_time_min"] = self.reading_time_min
        parsed_story_dict["story_cats"] = self.soup.find_all(self.__get_categories)
        parsed_story_dict["story_tags"] = self.soup.find_all(self.__get_tags)
        parsed_story_dict["story_uuid"] = self.story_uuid
        parsed_story_dict["story_rating"], parsed_story_dict["story_votes"] = self.__class__.get_ratings_and_votes(self.story_uuid)
        return parsed_story_dict

    @classmethod
    def get_ratings_and_votes(cls, story_uuid):
        response = requests.post(
            'https://www.creepypasta.com/wp-admin/admin-ajax.php',
            params=request_args["params"],
            headers=request_args["headers"],
            data=get_data_arg(story_uuid),
        )

        rating_regex_2 = r"<strong>([0-9\.]*)<\/strong>\/10\.\s*From\s*([0-9]*.*[0-9]*K*)\s*votes."
        rating_render_text = response.json()["items"][0]["render"]
        rating_groups = re.search(rating_regex_2, rating_render_text, flags=re.IGNORECASE)
        avg_rating, num_votes = rating_groups.group(1), rating_groups.group(2)
        if "k" in num_votes.lower():
            num_votes_groups = re.search(r"([0-9]*\.*[0-9]*)\S*\s*K", num_votes, flags=re.IGNORECASE)
            num_votes = int(float(num_votes_groups.group(1)) * 1000) if num_votes_groups else num_votes

        title_regex = r'"name":\s*"(.*)",\S*\s*"datePublished":'
        story_title = re.search(title_regex, rating_render_text, flags=re.IGNORECASE).group(1)
        print(f"'{story_title}' - avg rating: {avg_rating}/10, num votes: {num_votes}")
        return avg_rating, str(num_votes).strip()

    @property
    def story_link(self):
        return self.soup.find("link", {"rel": "canonical"})["href"]

    @property
    def story_uuid(self):
        return self.soup.select_one("article")["id"].strip("post-")

    @property
    def published_date(self):
        raw_date = self.soup.select_one("span.published").string.strip(" ")
        date_str = datetime.strftime(datetime.strptime(raw_date, "%B %d, %Y"), "%Y-%m-%d")
        return date_str

    @property
    def reading_time_min(self):
        reading_time_el = self.soup.select_one("span.rt-reading-time")
        reading_time_raw_unit = reading_time_el.select_one("span.rt-postfix").string
        reading_time_raw_unit

        reading_time_raw = int(reading_time_el.select_one("span.rt-time").string.strip(" "))
        reading_time_min = reading_time_raw if re.search(r"minutes*", reading_time_raw_unit) else (reading_time_raw * 60)
        return reading_time_min

    @property
    def story_content(self):
        return self.__get_story_and_author_content()[0]
    
    @property
    def author_content(self):
        return self.__get_story_and_author_content()[1]

    def __get_story_and_author_content(self):
        entry_content = self.soup.select('div.entry-content > p')
        if hasattr(entry_content[-1], "text"):
            if "credit" in entry_content[-1].text.lower():
                story_content, author_str = entry_content[:-1], entry_content[-1].text
            else:
                story_content, author_str = entry_content, None
        else:
            if "credit" in entry_content[-1].string.lower():
                story_content, author_str = entry_content[:-1], entry_content[-1].string
            else:
                story_content, author_str = entry_content, None  

        story_content = [(re.sub("<p>", "", p.text)) for p in story_content]

        if author_str:
            raw_author_regex = r"Credit[ed]*\s*[to]*\s:*\s*(\w*\s*\w*)\.*"
            author_str = re.search(
                raw_author_regex,
                author_str,
                re.IGNORECASE).group(1)
        return story_content, author_str

    def __get_categories(self, tag):
        return tag.has_attr('href') \
            and re.search(r"category", tag["href"]) \
            and not re.search(r"archive", tag["href"])

    def __get_tags(self, tag):
        return tag.has_attr('href') \
            and re.search(r"tag", tag["href"]) \
            and not re.search(r"instagram", tag["href"])

In [902]:
test_story_1 = "./story-pages/published_stories_contamination.html"
test_story_2 = "./story-pages/published_stories_holder-of-eternity.html"
test_story_3 = "./story-pages/published_stories_the-kaleidoscope.html"
test_story_4 = "./story-pages/published_stories_the-town-of-blanche.html"

# parser = CreepyPastaSoupParser(test_story_1, is_fpath=True)
parser = CreepyPastaSoupParser("https://www.creepypasta.com/gift-of-old/", is_fpath=False)
parser.parse_story()
# parser.story_content

Fetching https://www.creepypasta.com/gift-of-old/
'Gift of Old' - avg rating: 6.76/10, num votes: 58 


{'title': 'Gift of Old - Creepypasta',
 'link': 'https://www.creepypasta.com/gift-of-old/',
 'date_str': '2023-07-11',
 'story_text': ['A young, dark blond woman with a backpack checked the list of stores in her hand. Of the seven names, she crossed out four. She checked her phone for directions to the antique store Gifts of Old. She put her phone away and read the back of the paper, a set of instructions she should follow if she ever encountered a demon or spirit. Never tell them your name. Always say goodbye. Keep your words short and to the point, or they’ll manipulate what you say. Do not accept any offers.',
  'The young woman pulled a picture of her and another blond woman out of her bag. She pressed it to her heart, then tucked it away. A bell rang as she pushed the antique store’s door open. A woman at the counter shifted her gaze from the objects before her to the new customer.',
  'The store owner stood up straighter as the woman approached. “Good morning.” The store clerk sp

In [903]:
parser.published_date

'2023-07-11'

# BeautifulSoup for Ratings and Reviews
NEED TO USE UNDOC API

In [904]:
import requests

from bs4 import BeautifulSoup

In [905]:
def get_story_and_author_content(soup):
    entry_content = soup.select('div.entry-content > p')
    if hasattr(entry_content[-1], "text"):
        if "credit" in entry_content[-1].text.lower():
            story_content, author_str = entry_content[:-1], entry_content[-1].text
        else:
            story_content, author_str = entry_content, None
    else:
        if "credit" in entry_content[-1].string.lower():
            story_content, author_str = entry_content[:-1], entry_content[-1].string
        else:
            story_content, author_str = entry_content, None  

    story_content = [(re.sub("<p>", "", p.text)) for p in story_content]

    if author_str:
        raw_author_regex = r"Credit[ed]*\s*[to]*\s:*\s*(\w*\s*\w*)\.*"
        author_str = re.search(
            raw_author_regex,
            author_str,
            re.IGNORECASE).group(1)
    return story_content, author_str

In [906]:
page_html = get_html(test_story_2)
soup = BeautifulSoup(page_html)
soup.title
get_story_and_author_content(soup)

(['In any city, in any country, go to any mental institution or you can get yourself to. When you reach the front desk, ask to visit someone who calls himself “The Holder of Eternity”. A sigh might escape the worker as they look upon you with the utmost pity. They will take you down a flight of stairs into what should be the basement of the building, and yet isn’t.',
  'As you press deeper and deeper into this under layer of the institution a chorus of screams will begin to be audible. Softly at first, as if from a great distance, but the closer you get to the end of the hallway, the louder it becomes until it drones so loud that it seems to consume all other noises, until you begin to claw at your own ears in pain. The worker will show you a door, covering both their ears. As swift as they can, they will unlock the door and run, leaving only you in this cramped, dark hallway.',
  'This is your last chance to run. If you decide to continue, and open the door then the piercing wail will

In [907]:
request_args = {
    "headers": {
        'authority': 'www.creepypasta.com',
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'accept-language': 'en-US,en;q=0.9',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'origin': 'https://www.creepypasta.com',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
        'sec-ch-ua-mobile': '?1',
        'sec-ch-ua-platform': '"Android"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36',
        'x-requested-with': 'XMLHttpRequest',
    },
    "params": {
        'action': 'gdrts_live_handler',
    }
}

def get_data_arg(story_id=210):
    data = {
        'req': '{"todo":"dynamic","items":[{"args":{"echo":false,"entity":"posts","name":"post","item_id":0,"id":' 
            + str(story_id) 
            + ',"method":"stars-rating","series":null},"method":{"votes_count_compact_show":true,"votes_count_compact_decimals":"2","cta":"","distribution":"normalized","rating":"average","style_name":"star", "labels":[]},"did":1},{"args":{"echo":false,"entity":"posts","name":"post","item_id":null,"id":' + str(story_id) + ',"method":"stars-rating","series":null,"disable_dynamic_load":false,"dynamic":true},"method":{"votes_count_compact_show":true,"votes_count_compact_decimals":"2","cta":"","distribution":"normalized","rating":"average","labels":[]},"did":2}]}'
    }
    return data

def get_rating_and_votes_inner(response):
    rating_regex_2 = r"<strong>([0-9\.]*)<\/strong>\/10\.\s*From\s*([0-9]*.*[0-9]*K*)\s*votes."
    rating_render_text = response.json()["items"][0]["render"]
    rating_groups = re.search(rating_regex_2, rating_render_text, flags=re.IGNORECASE)
    avg_rating, num_votes = rating_groups.group(1), rating_groups.group(2)
    if "k" in num_votes.lower():
        num_votes_groups = re.search(r"([0-9]*\.*[0-9]*)\S*\s*K", num_votes, flags=re.IGNORECASE)
        num_votes = (float(num_votes_groups.group(1)) * 1000) if num_votes_groups else num_votes

    title_regex = r'"name":\s*"(.*)",\S*\s*"datePublished":'
    story_title = re.search(title_regex, rating_render_text, flags=re.IGNORECASE).group(1)
    print(f"'{story_title}' - avg rating: {avg_rating}/10, num votes: {num_votes}")
    return avg_rating, num_votes

In [908]:
def get_ratings_and_votes(uuid):
    response = requests.post(
        'https://www.creepypasta.com/wp-admin/admin-ajax.php',
        params=request_args["params"],
        headers=request_args["headers"],
        data=get_data_arg(uuid),
    )

    return get_rating_and_votes_inner(response)

In [909]:
get_ratings_and_votes(225)

'The X-Ray Glasses' - avg rating: 5.27/10, num votes: 215 


('5.27', '215 ')

In [910]:
def parse_story(page_html):
    soup = BeautifulSoup(page_html, "lxml")
    # title = soup.select_one(".entry-title").contents[0]
    parsed_story_dict = {}
    parsed_story_dict["title"] = soup.title.string
    parsed_story_dict["link"] = soup.find("link", {"rel": "canonical"})["href"]
    parsed_story_dict["date_str"] = get_date(soup)
    parsed_story_dict["story_text"], parsed_story_dict["author_str"] = get_story_and_author_content(soup)
    parsed_story_dict["reading_time_min"] = get_reading_time_min(soup)
    parsed_story_dict["story_cats"] = soup.find_all(get_categories)
    parsed_story_dict["story_tags"] = soup.find_all(get_tags)
    parsed_story_dict["story_uuid"] = get_story_uuid(soup)
    parsed_story_dict["story_rating"], parsed_story_dict["story_votes"] = get_ratings_and_votes(parsed_story_dict["story_uuid"])
    return parsed_story_dict

In [911]:
def get_reading_time_min(soup):
    reading_time_el = soup.select_one("span.rt-reading-time")
    reading_time_raw_unit = reading_time_el.select_one("span.rt-postfix").string
    reading_time_raw_unit

    reading_time_raw = reading_time_el.select_one("span.rt-time").string.strip(" ")
    if (re.search(r"\D", reading_time_raw)):
        return 0.5 # stand-in for < 1 min
    reading_time_min = int(reading_time_raw) if re.search(r"minutes*", reading_time_raw_unit) else (int(reading_time_raw) * 60)
    return reading_time_min

In [912]:
test_story_1 = "./story-pages/published_stories_contamination.html"
test_story_2 = "./story-pages/published_stories_the-x-ray-glasses.html"
test_story_3 = "./story-pages/published_stories_the-kaleidoscope.html"
test_story_4 = "./story-pages/published_stories_the-town-of-blanche.html"
page_html = get_html(test_story_4)
soup = BeautifulSoup(page_html, "lxml")
get_reading_time_min(soup)
get_story_and_author_content(soup)

(['If you visit France’s Cote d’Azur in your lifetime, do everything you can to avoid a small town called “Blanche”. I was in the country once with my parents, around 8 years ago (I was 12 at that time; we were on a family vacation), and we found ourselves looking for a place to get some rest and enjoy some of the local color. We were getting really hungry on the road, so it was with some luck that a town, unmarked on our map, rose unexpectedly on the horizon. This was the town of Blanche.',
  'Immediately after we entered Blanche, we noticed that the colors of the houses were darker than anything I had seen in my entire life – it’s not like they were black or grey, they were normal colors for walls – they just look… not right. It’s hard to explain; almost like it was a color that we don’t even have a word for because it’s so dark and strange.',
  'A few minutes after driving around the town, we all began to notice the fishy stench – like a Friday market, except for the fact that no fi

In [913]:
temp_list = [parse_story(page_html),]

'The Town Of Blanche' - avg rating: 2.63/10, num votes: 3760.0


In [914]:
import pandas as pd

In [915]:
df = pd.DataFrame(temp_list)
df

,title,link,date_str,story_text,author_str,reading_time_min,story_cats,story_tags,story_uuid,story_rating,story_votes
0,The Town Of Blanche - Creepypasta,https://www.creepypasta.com/the-town-of-blanche/,2008-09-29,[If you visit France’s Cote d’Azur in your lif...,None,2,[[Locations and Sites]],[],212,2.63,3760.0


In [917]:
def get_story_json_from_fpath(fpath):
    page_html = get_html(fpath)
    return parse_story(page_html)

def get_story_json_from_slug(slug):
    pass

In [918]:
get_story_json_from_fpath(test_story_2)

'The X-Ray Glasses' - avg rating: 5.27/10, num votes: 215 


{'title': 'The X-Ray Glasses - Creepypasta',
 'link': 'https://www.creepypasta.com/the-x-ray-glasses/',
 'date_str': '2008-10-11',
 'story_text': ['For a brief period in 1971, a New Jersey based company sold novelty “X-ray” glasses through the mail via advertisements in the Marvel line of comic books. People who viewed their televisions while wearing these glasses reported seeing images that were “hellish” or “like hell”. It should be noted that this phenomena occurred whether the televisions in question were turned on or not. The company quickly went out of business and investigations reveal that the company’s address leads to a graveyard founded over 150 years earlier.',
  '—\nEDIT: Closing the comments on this one for now, as it seems this particular entry got on some lame spammer’s hit list and they’re leaving like 50 spam comment attempts per day on this entry and I’m getting sick of it! You’d think they’d figure out, after hundreds of tries, that the comments here are moderated a

In [919]:
stories_path = "./story-pages/"
for fpath in get_all_files(stories_path):
    if fpath == ".DS_Store":
        continue
    get_story_json_from_fpath(stories_path + fpath)

'The X-Ray Glasses' - avg rating: 5.27/10, num votes: 215 
'Olfactory' - avg rating: 6.47/10, num votes: 255 
'The Kaleidoscope' - avg rating: 8.77/10, num votes: 560 
'The Seven Springs' - avg rating: 5.85/10, num votes: 210 
'The Forgotten Vending Machine' - avg rating: 6.22/10, num votes: 342 
'The Hooded Being' - avg rating: 6.32/10, num votes: 207 
'The Lost Cavern City' - avg rating: 4.19/10, num votes: 205 
'In The Mirror' - avg rating: 8.32/10, num votes: 323 
'Thanks' - avg rating: 8.38/10, num votes: 627 
'Contamination' - avg rating: 7.45/10, num votes: 273 
'Gloomy Sunday' - avg rating: 5.06/10, num votes: 212 
'Holder of Eternity' - avg rating: 7.63/10, num votes: 318 
'The Photograph Pile' - avg rating: 8.56/10, num votes: 417 
'The Town Of Blanche' - avg rating: 2.63/10, num votes: 3760.0
'The Church Cellar' - avg rating: 7.18/10, num votes: 279 
'A Chat Over Dinner' - avg rating: 7.83/10, num votes: 306 
'The 50&#8217;s Restaurant' - avg rating: 5.60/10, num votes: 286 